In [1]:
import pandas as pd
import numpy as np

## Generate a ground truth pmat

In [72]:

n_rows = 6 #total number of mutations in pmat
clones = 6 # number of clones
samples = 6 # number of cells per clone
n_mutations = 1 # number of mutations per clone
high_prob = 0.9 # probability for presence of mutation
low_prob = 0.0001 # probability for absence of mutation

if n_rows != n_mutations * clones:
    print('number of mutations per clone is not matching the total number of mutations!')

# clone name
c = [f'c{w+1}' for w in range(clones)]

# cell name
s = [f's{w+1}' for w in range(samples)]

# multi index by product
mux = pd.MultiIndex.from_product([c, s])

#Generate a dataframe with with default value
gt = pd.DataFrame(low_prob, index=np.arange(n_rows) // n_mutations, columns=mux)

# random assign high probability for mutation by random index with no repeat
for c, i in zip(gt.columns.levels[0], 
                np.random.choice(np.unique(gt.index), clones, replace=False)):
    gt.loc[i, c] = high_prob

gt.columns = gt.columns.map(lambda clones: f'{clones[0]}_{clones[1]}')

gt = gt.reset_index(drop=True)

# add bulk
gt.insert(loc=0, column='bulk', value=0.0001)
#gt['bulk'] = 0.0001

In [74]:
gt.shape

(6, 37)

In [75]:
exp_name = '0'
matrix_path = '../../data/simulated_matrix_output/' + exp_name + '.csv'
gt.to_csv(matrix_path, index=False, sep=' ', header= False)

## Remove clone specific mutations

In [78]:
rm_1 = gt.copy()
rm_1 = rm_1.drop([1])
rm_2 = rm_1.drop([2])
rm_3 = rm_2.drop([3])
rm_4 = rm_3.drop([4])

matrix_path = '../../data/simulated_matrix_output/1.csv'
rm_1.to_csv(matrix_path, index=False, sep=' ', header= False)

matrix_path = '../../data/simulated_matrix_output/2.csv'
rm_2.to_csv(matrix_path, index=False, sep=' ', header= False)

matrix_path = '../../data/simulated_matrix_output/3.csv'
rm_3.to_csv(matrix_path, index=False, sep=' ', header= False)

matrix_path = '../../data/simulated_matrix_output/4.csv'
rm_4.to_csv(matrix_path, index=False, sep=' ', header= False)

## Add non informative mutations

In [92]:
cells_w_noise = [2, 4, 8, 16, 32]

for n in cells_w_noise:
    rm_1_w_noise = rm_1.copy().T
    rm_1_w_noise['noise1'] = low_prob # add a position with default value for low probability
    noisy_samples = rm_1_w_noise.sample(n) 
    noisy_samples.noise1=0.9
    rm_1_w_noise.update(noisy_samples)
    update_list = noisy_samples.index.tolist() # save noise samples in list for checking
    n = str(n)
    matrix_path = '../../data/simulated_matrix_output/rm1_w_1_noisy_pos_' + n + '_noisy_samples.csv'
    rm_1_w_noise.to_csv(matrix_path, index=False, sep=' ', header= False)